In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/dataset_full.zip -d /content/dataset_1

Archive:  /content/drive/MyDrive/dataset_full.zip
   creating: /content/dataset_1/dataset_full/Building/
  inflating: /content/dataset_1/dataset_full/Building/0.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10014.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10144.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10191.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10210.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10228.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10251.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10312.jpg  
  inflating: /content/dataset_1/dataset_full/Building/1041.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10444.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10480.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10485.jpg  
  inflating: /content/dataset_1/dataset_full/Building/10489.jpg  
  inflating: /content/dataset_1/dataset_fu

In [37]:
!pip install mahotas
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score
from skimage.exposure import histogram



#importing pipeline
from sklearn.pipeline import Pipeline

import mahotas

# import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [54]:
data_path = r'/content/dataset_1/dataset_full'
labels = os.listdir(data_path)

x_image = []
y_label = []

for dirname in labels:
    filepath = os.path.join(data_path, dirname)
    for file in os.listdir(filepath):
        filename = os.path.join(filepath, file)
        image = cv2.imread(filename)
        image_resized = cv2.resize(image, (300,300))
        image_gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
        # HUMoments for shape
        image_hu = cv2.HuMoments(cv2.moments(image_gray)).flatten()
        # Haralick for texture
        image_har = mahotas.features.haralick(image_gray).mean(axis=0)
        # convert the image to HSV color-space
        image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        # color histogram
        image_hist  = cv2.calcHist([image_hsv], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        # cv2.normalize(image_hist, image_hist)
        image_hist_flat = image_hist.flatten()
        # Histogram
        # hist, hist_centers = histogram(image_resized)
        # SIFT features
        sift = cv2.SIFT_create()
        kp, des = sift.detectAndCompute(image_resized, None)
        if des is not None:
            sift_features = des.flatten()
            # Ensure the length is fixed
            if len(sift_features) < 128:
                sift_features = np.pad(sift_features, (0, 128 - len(sift_features)), mode='constant')
            else:
                sift_features = sift_features[:128]
        else:
          sift_features = np.zeros(128)
        # combine the features extracted
        f_vector_concat = np.hstack([image_hist_flat, image_har, image_hu,sift_features])
        # print(len(f_vector_concat))
        x_image.append(f_vector_concat)
        y_label.append(dirname)
        continue
    continue
    break

In [56]:
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, precision_score, recall_score, accuracy_score, f1_score

x_image_n = np.array(x_image)
y_label_n = np.array(y_label)
le = LabelEncoder()
y_label_le = le.fit_transform(y_label)
print(y_label_le)

# feature scaling between 0-1
scaler = MinMaxScaler(feature_range=(-1, 1))
x_image_scaled = scaler.fit_transform(x_image_n)

# #split data set
x_train, x_test, y_train, y_test = train_test_split(x_image_n, y_label_le, test_size=0.2, random_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)



[5 5 5 ... 2 2 2]
(2515, 660)
(629, 660)
(2515,)
(629,)


In [5]:
#train the model
clf_rfc = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=9,max_features='sqrt', subsample=0.8,random_state=42).fit(x_train, y_train)
# evaluate the model
y_pred_rfc = clf_rfc.predict(x_test)
print("accuracy_score :-  ", accuracy_score(y_test, y_pred_rfc))
print("precision_score:-  ",precision_score(y_test, y_pred_rfc, average='weighted', labels=np.unique(y_pred_rfc)))
print("confusion_matrix :- ",confusion_matrix(y_test, y_pred_rfc))
print("f1_score :- " ,f1_score(y_test, y_pred_rfc, average='weighted', labels=np.unique(y_pred_rfc)))


accuracy_score :-   0.8058823529411765
precision_score:-   0.802734175683394
confusion_matrix :-  [[ 55   4   8   5  14  28]
 [  1 111   1   4   1   4]
 [  5   0  71  10   5   0]
 [  2   4  11  82   6   2]
 [ 15   1  12  10  57   7]
 [ 16   3   0   2   4  68]]
f1_score :-  0.8016612957237466


In [4]:
# create a list of models to evaluate

models = [
    ('Logistic Regression', LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=1.0,random_state=42)),
    ('Gradient Boosting', GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=9,max_features='sqrt', subsample=0.8,random_state=42)),
    ('KNeighbors Classifier', KNeighborsClassifier(n_neighbors=5)),
    ('Decision Tree Classifier', DecisionTreeClassifier(random_state=42)),
    ('AdaBoost Classifier', AdaBoostClassifier(random_state=42)),
    ('Random Forest', RandomForestClassifier(random_state=42)),
    ('XGboost Classifier', XGBClassifier(random_state=42)),

    ('Support Vector Machine', SVC(random_state=42)),

    ('Naye base Classifier', GaussianNB())


]

best_model = None
best_accuracy = 0.0

#Iterate over the models and evaluate their performance
for name, model in models:
    #create a pipeline for each model
    pipeline = Pipeline([
        # ('imputer', SimpleImputer(strategy='most_frequent)),
        #('encoder', OneHotEncoder(handle_unknow='ignore'))
        ('model', model)
    ])

    # perform cross validation
    scores = cross_val_score(pipeline, x_image_n, y_label_le, cv=5)

    # Calculate mean accuracy
    mean_accuracy = scores.mean()

    #fit the pipeline on the training data
    pipeline.fit(x_train, y_train)

    # make prediction on the test data
    y_pred = pipeline.predict(x_test)

    #Calculate accuracy score
    accuracy = accuracy_score(y_test, y_pred)

    #print the performance metrics
    print("Model", name)
    print("Cross Validatino accuracy: ", mean_accuracy)
    print("Test Accuracy: ", accuracy)
    print()

    #Check if the current model has the best accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = pipeline

# Retrieve the best model
print("Best Model: ", best_model)

Model Logistic Regression
Cross Validatino accuracy:  0.5866454689984102
Test Accuracy:  0.5643879173290938

Model Gradient Boosting
Cross Validatino accuracy:  0.86332273449921
Test Accuracy:  0.88332273449921

Model KNeighbors Classifier
Cross Validatino accuracy:  0.50717011128776
Test Accuracy:  0.52992050874403816

Model Decision Tree Classifier
Cross Validatino accuracy:  0.5664387917329
Test Accuracy:  0.571542130365659

Model AdaBoost Classifier
Cross Validatino accuracy:  0.8707472178060413
Test Accuracy:  0.8907472178060413

Model Random Forest
Cross Validatino accuracy:  0.6431333731633471
Test Accuracy:  0.6836248012718601

Model XGboost Classifier
...

Best Model:  Pipeline(steps=[('model',
                 XGBClassifier(objective='multi:softmax',eval_metric=['merror','mlogloss']
                                ,early_stopping_rounds=10))])


In [3]:
categorical_cols = ['Building', 'Sea', 'Streets', 'Forest', 'Glacier', 'Mountains']
def evaluate_classification_models(x_image_n, y, categorical_columns):

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(x_image_n, y, test_size=0.2, random_state=0)

    # Define models
    models = {
        "Logistic Regression": LogisticRegression(),
        "KNN": KNeighborsClassifier(),
        "NB": GaussianNB(),
        "SVM": SVC(),
        "Decision Tree": DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(),
        "XGBoost": XGBClassifier(),
        "GradientBoosting": GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=9,max_features='sqrt', subsample=0.8,random_state=42),
        "AdaBoost": AdaBoostClassifier()
    }

    # Train and evaluate models
    results = {}
    best_model = None
    best_accuracy = 0.0
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results[name] = accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = name

    return results, best_model

# Example usage:
results, best_model = evaluate_classification_models(x_image_scaled, y_label_le, categorical_cols)
import pickle
print("Model accuracies:", results)
print("Best model:", best_model)
file_name = f"/content/sample_data/{best_model}.pkl"

# save
pickle.dump(best_model, open(file_name, "wb"))


Model accuracies: {'Logistic Regression': 0.5866454689984102, 'KNN': 0.50717011128776, 'NB': 0.63306836248, 'SVM': 0.723370429252782, 'Decision Tree': 0.5064387917329, 'Random Forest': 0.6836248012718601, 'XGBoost': 0.90925278219396, 'GradientBoosting': 0.86332273449921, 'AdaBoost': 0.8707472178060413}
Best model: XGBoost


In [1]:
def hyperparameter_tuning(X, y, categorical_columns, models):
    # Define dictionary to store results
    results = {}

    # Encode categorical columns
    # X_encoded = X.copy()
    # label_encoders = {}
    # for col in categorical_columns:
    #     label_encoders[col] = LabelEncoder()
    #     X_encoded[col] = label_encoders[col].fit_transform(X[col])

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    # Perform hyperparameter tuning for each model
    for model_name, model in models.items():
    # Define parameter grid for hyperparameter tuning
        param_grid = {}
    if model_name == 'Logistic Regression':
        param_grid = {'C': [0.1, 1, 10, 100]}
    elif model_name == 'KNN':
        param_grid = {'n_neighbors': [3, 5, 7, 9],"weights" : ['uniform', 'distance'],"algorithm": ['auto', 'ball_tree', 'kd_tree','brute']}
    elif model_name == 'NB':
        param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]}
    elif model_name == 'SVM':
        param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10, 100]}
    elif model_name == 'Decision Tree':
        param_grid = {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    elif model_name == 'Random Forest':
        param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    elif model_name == 'XGBoost':
        param_grid = {'learning_rate': [0.01, 0.1, 0.2], 'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]}
    elif model_name == 'GradientBoosting':
        param_grid = {'learning_rate': [0.01, 0.1, 0.2], 'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]}
    elif model_name == 'AdaBoost':
        param_grid = {'learning_rate': [0.01, 0.1, 0.2], 'n_estimators': [50, 100, 200]}


        # Perform hyperparameter tuning using GridSearchCV
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
        grid_search.fit(X_train, y_train)

        # Get best hyperparameters and evaluate on test set
        best_params = grid_search.best_params_
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Store results in dictionary
        results[model_name] = {'best_params': best_params, 'accuracy': accuracy}

    return results

# Define models dictionary
models = {
        "Logistic Regression": LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=1.0,random_state=42),
        "KNN": KNeighborsClassifier(),
        "NB": GaussianNB(),
        "SVM": SVC(),
        "Decision Tree": DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(),
        "XGBoost": XGBClassifier(objective='multi:softmax',eval_metric=['merror','mlogloss'],early_stopping_rounds=10),
        "GradientBoosting": GradientBoostingClassifier(learning_rate=0.1, n_estimators=60,max_depth=9,max_features='sqrt', subsample=0.8,random_state=42),
        "AdaBoost": AdaBoostClassifier()
    }

# Example usage:
results = hyperparameter_tuning(x_image_n, y_label_le, categorical_cols, models)
for model_name, result in results.items():
    print("Model:", model_name)
    print("Best hyperparameters:", result['best_params'])
    print("Accuracy:", result['accuracy'])
    print()


Model: AdaBoost
Best hyperparameters: {'learning_rate': 0.2, 'n_estimators': 200}
Accuracy: 0.8945945945946
